In [1]:
import numpy as np
import pandas as pd
import regex as re
import spacy
import string

#!python -m spacy download en_core_web_sm

In [2]:
data = pd.read_csv('data\TA_restaurants_curated.csv')[['Reviews', 'Cuisine Style', 'City', 'Rating']]

In [3]:
data = data.drop(data.loc[data['Reviews'] == '[[], []]'].index).dropna()
data = data.drop(data.loc[data['Rating'] == -1].index).dropna()

In [4]:
def preproc_cuisine_style(data):
    data['Cuisine Style']=data['Cuisine Style'].fillna("missing values")
    data['Cuisine Style']=data['Cuisine Style'].str.replace("[","")
    data['Cuisine Style']=data['Cuisine Style'].str.replace("]","")
    data['Cuisine Style']=data['Cuisine Style'].apply(lambda x: x.split(','))
    data_preproc = pd.concat([data, pd.get_dummies(data['Cuisine Style'].apply(pd.Series), prefix='', prefix_sep='').sum(level=0, axis=1)],axis=1)
    data_preproc.pop('Cuisine Style')
    return data_preproc

def preproc_reviews(data):
    data['Reviews'] = data['Reviews'].apply(lambda x: re.findall('\[\[.*?\]', str(x)))
    data['Reviews'] = data['Reviews'].apply(lambda x: re.sub('\[\"\[\[\'', '', str(x)))
    data['Reviews'] = data['Reviews'].apply(lambda x: re.sub('\'\]\"]', '', str(x)))
    data[['Reviews_1', 'Reviews_2']] = data['Reviews'].str.split("', '", 1, expand=True)
    return data

def split_reviews(data):
    data_1 = data.drop(['Reviews', 'Reviews_2'], axis=1).rename(columns={'Reviews_1': 'Reviews'})
    data_2 = data.drop(['Reviews', 'Reviews_1'], axis=1).rename(columns={'Reviews_2': 'Reviews'})
    data_2 = data_2.drop(data_2.loc[data_2['Rating'] > 3.0].index)
    new_data = data_1.append(data_2).dropna()
    return new_data

def clean_text(data_text):
    text  = nlp_en_sm(data_text)
    string_2=''
    for token in text:
        string_2=string_2 +' '+ token.lemma_
    data_clean=string_2
    data_clean = data_clean.split()
    data_clean = [word for word in data_clean if not word in stop_words]
    data_clean = ' '.join(map(str, data_clean))
    return data_clean.strip(string.punctuation)

def create_categories(column):
    if column <= 3.0:
        return 0    
    elif column >=4.5:
        return 1
    else:
        return None

In [5]:
data = preproc_reviews(data)
data = split_reviews(data)
data = preproc_cuisine_style(data)
data['Category'] = data['Rating'].apply(lambda x: create_categories(x))

In [6]:
nlp_en_sm = spacy.load("en_core_web_sm")
stop_words = nlp_en_sm.Defaults.stop_words
data["Reviews_cleaned"] = data["Reviews"].apply(clean_text)
data = data.dropna()

In [7]:
data['Rating'].groupby(data['Category']).count()

Category
0.0    12582
1.0    28202
Name: Rating, dtype: int64

In [8]:
data.to_csv('data\preprocessed.csv')